<a href="https://colab.research.google.com/github/Diljitsingh14/Colab-Notebooks/blob/main/News-Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import matplotlib.dates as mdates
import re
from wordcloud import WordCloud
import seaborn as sns
import plotly.io as pio
import plotly.graph_objects as go
import math
import yfinance as yf
from datetime import datetime

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [2]:
import warnings
warnings.simplefilter("ignore")
np.random.seed(400)

In [4]:
start = datetime(2017,12,16)
end = datetime(2020,7,18)

df = yf.download("^DJI",start,end)
df.reset_index(inplace=True)
df.head()

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-12-18,24739.560547,24876.070312,24739.560547,24792.199219,24792.199219,354880000
1,2017-12-19,24834.380859,24850.109375,24715.599609,24754.750000,24754.750000,335330000
2,2017-12-20,24838.089844,24852.439453,24697.109375,24726.650391,24726.650391,341790000
3,2017-12-21,24778.259766,24850.910156,24766.269531,24782.289062,24782.289062,339530000
4,2017-12-22,24764.039062,24784.150391,24717.509766,24754.060547,24754.060547,255140000


In [5]:
!unzip archive\ \(6\).zip

Archive:  archive (6).zip
  inflating: cnbc_headlines.csv      
  inflating: guardian_headlines.csv  
  inflating: reuters_headlines.csv   


In [13]:
cnbc = pd.read_csv("cnbc_headlines.csv")
guardian = pd.read_csv("guardian_headlines.csv")
reuters = pd.read_csv("reuters_headlines.csv")

cnbc.head()

,Headlines,Time,Description
0,Jim Cramer: A better way to invest in the Covi...,"7:51 PM ET Fri, 17 July 2020","""Mad Money"" host Jim Cramer recommended buying..."
1,Cramer's lightning round: I would own Teradyne,"7:33 PM ET Fri, 17 July 2020","""Mad Money"" host Jim Cramer rings the lightnin..."
2,NaN,NaN,NaN
3,"Cramer's week ahead: Big week for earnings, ev...","7:25 PM ET Fri, 17 July 2020","""We'll pay more for the earnings of the non-Co..."
4,IQ Capital CEO Keith Bliss says tech and healt...,"4:24 PM ET Fri, 17 July 2020","Keith Bliss, IQ Capital CEO, joins ""Closing Be..."


In [14]:
concat_news_pdf = pd.concat([reuters,cnbc,guardian],ignore_index=True)
concat_news_pdf.tail()

,Headlines,Time,Description
53645,How investing in solar energy can create a bri...,17-Dec-17,NaN
53646,Poundland suppliers hit by insurance downgrade,17-Dec-17,NaN
53647,Cryptocurrencies: City watchdog to investigate...,17-Dec-17,NaN
53648,Unilever sells household name spreads to KKR f...,17-Dec-17,NaN
53649,The Guardian view on Ryanair’s model: a union-...,17-Dec-17,NaN


In [18]:
from dateutil import parser
def parse_date(input_data):
  try:
    date_obj = parser.parse(input_data)
  except ValueError:
    return None
  formatted_date = date_obj.strftime("%Y-%m-%d")
  return formatted_date
  # pass

In [19]:
concat_news_pdf['Time'] = concat_news_pdf['Time'].apply(lambda x: parse_date(x) if pd.notnull(x) else x)

In [21]:
features = concat_news_pdf[['Headlines','Time']]
features.head()

,Headlines,Time
0,TikTok considers London and other locations fo...,2020-07-18
1,Disney cuts ad spending on Facebook amid growi...,2020-07-18
2,Trail of missing Wirecard executive leads to B...,2020-07-18
3,Twitter says attackers downloaded data from up...,2020-07-18
4,U.S. Republicans seek liability protections as...,2020-07-17


In [22]:
features['Time'] = pd.to_datetime(features['Time'])

split_date = pd.to_datetime('2020-01-01')

train_df = features[features['Time'] < split_date]
test_df = features[features['Time'] >= split_date]

In [23]:
len(train_df), len(test_df)

(39463, 13907)

In [24]:
features.iloc[32770:32774]

,Headlines,Time
32770,Jim Cramer: A better way to invest in the Covi...,2020-07-17
32771,Cramer's lightning round: I would own Teradyne,2020-07-17
32772,NaN,NaT
32773,"Cramer's week ahead: Big week for earnings, ev...",2020-07-17


In [25]:
def remove_row_with_nan(features,column_name):
  features_filterd = features.dropna(subset=[column_name])
  return features_filterd

In [29]:
features  = remove_row_with_nan(features,'Headlines')
features.iloc[32770:32774]

,Headlines,Time
32770,Jim Cramer: A better way to invest in the Covi...,2020-07-17
32771,Cramer's lightning round: I would own Teradyne,2020-07-17
32773,"Cramer's week ahead: Big week for earnings, ev...",2020-07-17
32774,IQ Capital CEO Keith Bliss says tech and healt...,2020-07-17


In [30]:
from transformers import pipeline
from transformers import AutoTokenizer,AutoModelForSequenceClassification
from collections import defaultdict
from tqdm import tqdm

In [31]:
pipe = pipeline('text-classification',model='ProsusAI/finbert')
tokenizer = AutoTokenizer.from_pretrained('ProsusAI/finbert')

model = AutoModelForSequenceClassification.from_pretrained('ProsusAI/Finbert')

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [32]:
def analyze_sentiment(dataframe,datelabel,textlabel):
  sentiments = defaultdict(lambda : {'pos':[],'neg':[],'neu':[]})

  for index, row in tqdm(dataframe.iterrows(),total=len(dataframe)):
    text = [row[textlabel]]

    inputs = tokenizer(text,return_tensors='pt')

    outputs = model(**inputs)

    probs = outputs.logits.softmax(dim=1)

    pos = probs[:,0].tolist()
    neg = probs[:,1].tolist()
    neu = probs[:,2].tolist()

    sentiments[row[datelabel]]['pos'].extend(pos)
    sentiments[row[datelabel]]['neg'].extend(neg)
    sentiments[row[datelabel]]['neu'].extend(neu)
  return sentiments

In [ ]:
sentiments = analyze_sentiment(features,'Time','Headlines')
sentiments

 37%|███▋      | 19708/53370 [28:14<46:10, 12.15it/s]